In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
import pip
pip.main(['install','seaborn'])
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from keras.applications import DenseNet121, VGG19
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, LeakyReLU, GlobalAveragePooling2D, Conv2D, Input
from keras.optimizers import Adam
from keras.losses import BinaryFocalCrossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

modelclass = '3'
suffix = 'FIN2'
iterations = 3
iterations2 = 3

def CreateModel(learn_rate):
    input_tensor = Input(shape=(256, 256, 3))
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    for layer in base_model.layers[:10]:
        layer.trainable = False
        
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learn_rate),
                  loss=BinaryFocalCrossentropy(),
                  metrics=['accuracy'])

    return model

base_dir = '../../../FoundationNIH/Data/Arrays/'
class_dir = 'Converted Arrays/'
train_images = np.load(os.path.join(base_dir, 'train_images.npy'))
train_labels = np.load(os.path.join(class_dir, f'train_labels_class{modelclass}.npy'))
val_images = np.load(os.path.join(base_dir, 'val_images.npy'))
val_labels = np.load(os.path.join(class_dir, f'val_labels_class{modelclass}.npy'))

if len(train_images.shape) == 3:
    train_images = np.expand_dims(train_images, axis=-1)
    train_images = np.repeat(train_images, 3, axis=-1)

if len(val_images.shape) == 3:
    val_images = np.expand_dims(val_images, axis=-1)
    val_images = np.repeat(val_images, 3, axis=-1)

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels.flatten())
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

print("Initial class weights:", class_weights)
print("Start time: ", datetime.datetime.now())

best_model_path = f'Models/class{suffix}{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')

best_val_loss = float('inf')
for i in range(iterations):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-7)
    model = CreateModel(1e-5)
    print(f"Training iteration {i+1}/{iterations}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbest.keras')

print("Training Complete")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: seaborn in /opt/conda/lib/python3.11/site-packages (0.13.2)

Requirement already satisfied: numpy!=1.24.0,>=1.20 in /opt/conda/lib/python3.11/site-packages (from seaborn) (1.26.4)

Requirement already satisfied: pandas>=1.2 in /opt/conda/lib/python3.11/site-packages (from seaborn) (2.2.3)

Requirement already satisfied: matplotlib!=3.6.1,>=3.4 in /opt/conda/lib/python3.11/site-packages (from seaborn) (3.9.3)

...

  1/354 ━━━━━━━━━━━━━━━━━━━━ 2:05:50 21s/step - accuracy: 0.7500 - loss: 0.9499

I0000 00:00:1736949030.163004   14227 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


353/354 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.5072 - loss: 0.3038

E0000 00:00:1736949102.197887   14227 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1736949102.419281   14227 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1736949103.839090   14227 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1736949104.138624   14227 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


354/354 ━━━━━━━━━━━━━━━━━━━━ 123s 288ms/step - accuracy: 0.5072 - loss: 0.3033 - val_accuracy: 0.4488 - val_loss: 0.1746 - learning_rate: 1.0000e-05
Epoch 2/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 73s 207ms/step - accuracy: 0.4839 - loss: 0.1749 - val_accuracy: 0.4576 - val_loss: 0.1750 - learning_rate: 1.0000e-05
Epoch 3/50
102/354 ━━━━━━━━━━━━━━━━━━━━ 48s 191ms/step - accuracy: 0.4625 - loss: 0.1739

In [ ]:
best_model_path = f'Models/class{suffix}secondary{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')
best_val_loss = float('inf')
for i in range (iterations2):
    model = CreateModel(1e-7)
    model.load_weights(f'Models/class_{suffix}{modelclass}_overallbest.keras')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)
    
    print(f"Training iteration {i+1}/{iterations2}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best last val:{val_loss}")
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbestFIN.keras')

In [ ]:

test_images = np.load(os.path.join(base_dir, 'test_images.npy'))
test_labels = np.load(os.path.join(class_dir, f'test_labels_class{modelclass}.npy'))
test_image_names = np.load(os.path.join(base_dir, 'test_image_names.npy'), allow_pickle=True)

if len(test_images.shape) == 3:
    test_images = np.expand_dims(test_images, axis=-1)
    test_images = np.repeat(test_images, 3, axis=-1)

test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=1)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

y_pred = model.predict(test_images)
y_pred_binary = (y_pred > 0.5).astype(int)

cm = confusion_matrix(test_labels, y_pred_binary)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

def display_image_with_predictions(image, image_name, actual_label, predicted_label, likelihood):
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"Image: {image_name}\nActual: {actual_label}\nPredicted: {predicted_label}\nLikelihood: {likelihood:.4f}")
    plt.axis('off')
    plt.show()

num_samples = 20
indices = random.sample(range(test_images.shape[0]), num_samples)

for idx in indices:
    image = test_images[idx]
    image_name = test_image_names[idx]
    actual_label = test_labels[idx][0]
    image_expanded = np.expand_dims(image, axis=0)
    prediction = model.predict(image_expanded)[0][0]
    
    threshold = 0.5
    predicted_label = int(prediction > threshold)
    likelihood = prediction
    
    display_image_with_predictions(image, image_name, actual_label, predicted_label, likelihood)